# Proyecto 2B: practicando el filtrado (también frecuencial)


## 1. Intuición acerca de las convoluciones bidimensionales y el filtrado

### 1.1 Operador de diferencias finitas

Se comenzará usando la diferencia finita en las direcciones $x$ y $y$: $D_x=(1 -1)$ y $D_y=
\begin{pmatrix}
1 \\ -1
\end{pmatrix}
$

Primero, muestre la derivada parcial en $x$ e $y$ de la imagen del camarógrafo (`skimage.data.camera()`) convolucionando la imagen con los operadores de diferencias finitas $D_x$ y $D_y$ (puede usar `scipy.signal.convolve2d`)). Ahora calcule y muestre la imagen de la magnitud del gradiente. Para convertir esto en una imagen de bordes, binarice la imagen de la magnitud del gradiente seleccionando un umbral apropiado (tratando de suprimir el ruido mientras se muestran todos los bordes reales; le tomará algunos intentos encontrar el umbral correcto).

### 1.2 Derivada del filtro Gaussiano (DoG)

Antes debió observar que los resultados con solo el operador de diferencia eran bastante ruidosos. Afortunadamente, tenemos un operador de suavizado: el filtro Gaussiano. Cree una versión borrosa de la imagen original convolucionando con un kernel gaussiano y repita el procedimiento de la parte anterior  ¿Qué diferencias ves? Ahora podemos hacer lo mismo con una sola convolución en lugar de dos creando una derivada de filtros gaussianos. Convolucione el kernel gaussiano con $D_x$ y $D_y$ y muestre los filtros DoG resultantes como imágenes. Verifique que el resultado sea el mismo que antes.

Una forma de crear un filtro gaussiano 2D es usando `scipy.signal.gaussian` para crear un gaussiano 1D y luego tomar un producto externo para obtener un kernel gaussiano 2D. Vea el código siguiente:

In [2]:
import numpy as np
from scipy import signal

def gkern(kernlen=21, std=3):
    """Returns a 2D Gaussian kernel array."""
    gkern1d = signal.gaussian(kernlen, std=std).reshape(kernlen, 1)
    gkern2d = np.outer(gkern1d, gkern1d)
    return gkern2d

### 1.3 Enderezamiento de una imagen

Recuerda la última vez que tomaste una foto con tu teléfono y la imagen no era recta. Probablemente intentaste girar la imagen manualmente para enderezarla. En este problema, se automatizará el proceso de enderezamiento de la imagen para ahorrar algo de tiempo. Se sabe que estadísticamente existe una preferencia por los bordes verticales y horizontales en la mayoría de las imágenes (¡debido a la gravedad!). Usaremos esta información al intentar rotar una imagen para maximizar el número de bordes verticales y horizontales. Para enderezar una imagen, genere un conjunto de rotaciones propuestas. Para cada ángulo de rotación propuesto:

- Gire la imagen según el ángulo de rotación propuesto utilizando la función de rotación incorporada en python (por ejemplo, scipy.ndimage.interpolation.rotate en python).
- Calcule el ángulo del gradiente de los bordes de la imagen (pero ignore los bordes de la imagen creados por la propia rotación; una forma sencilla de hacerlo es recortar siempre la parte central de la imagen).
- Calcule un histograma de estos ángulos (por ejemplo, matplotlib.pyplot.hist en python).

Finalmente, elija la rotación con el número máximo de bordes horizontales y verticales. Muestre el histograma de orientación y el resultado de enderezamiento de la siguiente imagen: ![](images2/facade.jpg)

También muestre su resultado en 3 imágenes más de su elección, de las cuales al menos una debería ser un caso de falla. Para cada imagen, debe mostrar la imagen original, la imagen enderezada y sus histogramas de orientación.
Pruebe a ver si puede mejorar su resultado tal vez usando el dominio de Fourier (esto es opcional extra).

## 2. Frecuencias

### 2.1 Nitidez de la imagen
Escoja una imagen borrosa y prepárese para "enfocarla".  Derive la técnica la técnica de enmascaramiento de enfoque (unsharp masking). El filtro Gaussiano, es un filtro pasabajas que retiene solo las frecuencias bajas. Podemos restar la versión borrosa de la imagen original para obtener las altas frecuencias de la imagen. Una imagen a menudo se ve más nítida si tiene altas frecuencias más fuertes. Entonces, agrega un poco más de frecuencias altas a la imagen. Combine esto en una sola operación de convolución que se denomina unsharp mask filter. Muestre su resultado en la siguiente imagen  más otras imágenes de su elección. ![](images2/taj.jpg)

También para propósistos de evaluación, elija una imagen nítida, difumínela (vuelvala borrosa) y luego intente enfocarla nuevamente. Compare la imagen original y la más nítida e informe sus observaciones.

### 2.2 Imágenes híbridas

#### Introducción 
El objetivo de esta parte de la tarea es crear [imágenes híbridas](http://olivalab.mit.edu/hybrid_gallery/gallery.html) utilizando el enfoque descrito en el [artículo](http://olivalab.mit.edu/publications/OlivaTorralb_Hybrid_Siggraph06.pdf) de Oliva, Torralba y Schyns. Las imágenes híbridas son imágenes estáticas que cambian de interpretación en función de la distancia de visualización. La idea básica es que la alta frecuencia tiende a dominar la percepción cuando está disponible, pero, a distancia, solo se puede ver la parte de baja frecuencia (suave) de la señal. Al combinar la parte de alta frecuencia de una imagen con la parte de baja frecuencia de otra, se obtiene una imagen híbrida que conduce a diferentes interpretaciones en diferentes distancias.

### Implementación
Se incluyen dos imágenes de muestra (gato y einstein) y un código para empezar de python que se puede usar para cargar dos imágenes y alinearlas (`hybrid_image_starter.py`). La alineación es importante porque afecta la agrupación perceptual (lea el paper para obtener más detalles).

1. Primero, necesitará obtener algunos pares de imágenes que desee convertir en imágenes híbridas. Puede usar las imágenes de muestra para depurar, pero debe usar sus propias imágenes en sus resultados. Luego, deberá escribir código para hacer un filtrado pasabajo sobre una imagen, hacer un filtrado pasaalto a la segunda imagen y agregar (o promediar) las dos imágenes. Para un filtro de pasabajo, Oliva et al. sugiera el uso de un filtro gaussiano 2D estándar. Para un filtro  pasaalto, sugieren usar el filtro de impulso menos el filtro gaussiano (que se puede calcular restando la imagen filtrada por Gauss de la original). La frecuencia de corte de cada filtro debe elegirse con cierta experimentación.

2. Para obtener su resultado favorito, también debe ilustrar el proceso a través del análisis de frecuencia. Muestre la magnitud logarítmica de la transformada de Fourier de las dos imágenes de entrada, las imágenes filtradas y la imagen híbrida. En python, puede calcular y mostrar la transformada de Fourier 2D con: 
`plt.imshow (np.log (np.abs (np.fft.fftshift (np.fft.fft2 (gris_ imagen)))))`.

3. Intente crear 2-3 imágenes híbridas (cambio de expresión, transformación entre diferentes objetos, cambio con el tiempo, etc.). Muestre la imagen de entrada y el resultado híbrido por cada ejemplo. (No es necesario mostrar los resultados intermedios como en el paso 2.)

Intente usar color para mejorar el efecto. ¿Funciona mejor usar color para el componente de alta frecuencia, el componente de baja frecuencia o ambos?

El código para ejecutar la imagen híbrida se muestra a continuación. En donde, se debe crear la función de `imagen_hibrida`. Nota: este script realiza la entrada de las coordenadas para alinear usando `ginput` que no funciona sobre jupyter, deberá cambiar este método por algo interactivo en el notebook como los widgets de jupyter.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib notebook
from alinear_imagen import align_images

# Carga las imagenes

# alta frecuencia
im1 = plt.imread('images2/einstein.jpg')/255.

# baja frecuencia
im2 = plt.imread('images2/gato.jpg')/255

# alinea las imagenes (pero puede mejorar)
im1_aligned, im2_aligned = align_images(im1, im2)

In [ ]:
## Deberas realizar el codigo de imagen_hibrida. Sigma1 and sigma2 son valores de corte arbitrarios para
## las bajas y altas frecuencias

sigma1 = arbitrary_value_1
sigma2 = arbitrary_value_2
hybrid = imagen_hibrida(im1, im2, sigma1, sigma2)

plt.imshow(hybrid)
plt.show

## Entregables

Para este proyecto debera entregar el desarrollo y visualización de los resultados en un notebook de jupyter donde se muestre el código realizado (y las pruebas). También, debe entregar un pequeño informe del proyecto con una breve descripción del proyecto explicando el enfoque desarrollado y el análisis de los resultados .(Puede ser otro notebook, pero sin código, o un documento en PDF).

Es aconsejable en el notebook solo mostrar las imágenes (resultados) en un formato comprimido (jpg por ejm).  Muestre los resultados mostrando las imagenes originales y las posteriores al procedimiento. 
Si algún resultado no es bueno, explique por qué sucede esto. Describa cualquier idea extra implementada. 